# Data Mining HW6
## Shardul Dabhane(sdabhane) B-565
### April 22nd, 2022

1. (40 points) Decision trees and ensemble approaches.

• Use sklearn’s breast cancer data set (from sklearn.datasets import load breast cancer)

• Try the bagging and adaboost approaches using the decision tree as the base
predictor. Experiment different parameters (e.g., number of base predictors).
You may use BaggingClassifier and AdaBoostClassifier in sklearn.ensemble
for this problem.

• Document what you have tried and report your results

In [1]:
#Import required modules
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

In [2]:
#We will now load the breast cancer dataset and convert it into a dataframe to perform operations
breast_cancer_data = load_breast_cancer()
df = pd.DataFrame(breast_cancer_data.data, columns=breast_cancer_data.feature_names)
df['target'] = pd.Series(breast_cancer_data.target)
df.head()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
# Separate the target and the feature attribute
features = df.columns[0:-1]
target = 'target'
X = df[features]
y = df[[target]]

# Use train test split to separate the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state=42, stratify=y)


In [4]:
#Set the base predictor to Decision Tree

dt=DecisionTreeClassifier()


In [5]:
#We will find the best parameters for bagging by passing a range of parameters 
#in the GridSearchCV function and using RepeatedKFold Cross Validation
finding_best_parameter_bagging_classifier = BaggingClassifier(base_estimator=dt)
parameters = {
'bootstrap_features': [True,False],
'max_features': [5,10,15],
'max_samples': [80,100,120],
'n_estimators': [50,75,100],
}
#Use GridSearchCV to get the best parameters that will give the best accuracy
search = GridSearchCV(finding_best_parameter_bagging_classifier, parameters, cv=RepeatedKFold(n_splits=10,n_repeats=3,random_state=42))

#Fit that model on the dataset
best_model = search.fit(X_train,y_train.values.ravel())

#Print the best accuracy and the parameters
print('The best accuracy score from this model is:')
print(best_model.best_score_)
print('The best hyperparameter configuration is:')
print(best_model.best_params_)

The best accuracy score from this model is:
0.9572845804988662
The best hyperparameter configuration is:
{'bootstrap_features': True, 'max_features': 5, 'max_samples': 120, 'n_estimators': 50}


In [6]:
#Call the bagging classifier function with the best parameters
bagging_classifier = BaggingClassifier(base_estimator=dt,max_features=5,max_samples=120,n_estimators=50,bootstrap=True,random_state=42)

#Fit that model on the dataset
bagging_classifier.fit(X_train, y_train)

#Print the coefficient of determination R^2 of the predictions on training and testing dataset
#A high score indicates that the model has good accuracy
print("The score on training data is",bagging_classifier.score(X_train, y_train))
print("The score on testing data is",bagging_classifier.score(X_test, y_test))

The score on training data is 0.9834368530020704
The score on testing data is 0.9418604651162791


/N/u/sdabhane/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [7]:
#We will find the best parameters for adaboost by passing a range of parameters 
#in the GridSearchCV function and using RepeatedKFold Cross Validation

finding_best_parameter_adaboost_classifier = AdaBoostClassifier(base_estimator=dt)

#Set the parameters
parameters = {
'learning_rate': [0.0005,0.005,0.05,0.5],
'n_estimators': [25, 75, 100]
}

#Use GridSearchCV to get the best parameters that will give the best accuracy
search = GridSearchCV(finding_best_parameter_adaboost_classifier, parameters, cv=RepeatedKFold(n_splits=10,n_repeats=3,random_state=42))

#Fit that model on the dataset
best_model = search.fit(X_train,y_train.values.ravel())

#Print the best accuracy and the parameters 
print('The best accuracy score from this model is:')
print(best_model.best_score_)
print('The best hyperparameter configuration is:')
print(best_model.best_params_)

The best accuracy score from this model is:
0.9248157596371881
The best hyperparameter configuration is:
{'learning_rate': 0.005, 'n_estimators': 100}


In [8]:
#Call the AdaBoostClassifier with the best parameters set from the previous cell
adaboost_classifier = AdaBoostClassifier(base_estimator=dt,learning_rate=0.005,n_estimators=100,random_state=42)

#Fit that model on the dataset
adaboost_classifier.fit(X_train, y_train)

#Print the coefficient of determination R^2 of the predictions on training and testing dataset:
#A high score indicates that the model has good accuracy
print("The score on training data is",adaboost_classifier.score(X_train, y_train))
print("The score on testing data is",adaboost_classifier.score(X_test, y_test))


The score on training data is 1.0
The score on testing data is 0.9302325581395349


/N/u/sdabhane/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


As we can observe here:

1. The best performance of BaggingClassifier with Decision Tree as the base predictor is when the following parameters are set:

**bootstrap_features=True, max_features=5, max_samples=120, n_estimators=50**

The BaggingClassifier gives us an accuracy of **0.9572845804988662** for the above parameters

2. The best performance of AdaBoostClassifier with Decision Tree as the base predictor is when the following parameters are set:

**learning_rate=0.005,n_estimators=100**

The AdaBoostClassifier gives us an accuracy of **0.9248157596371881** for the above parameters

3. Based on the given best configurations for each ensemble method, BaggingClassifier performs better compared to AdaBoostClassifier on the breast cancer dataset as it gives better accuracy.

References used in this code:

[1]. https://stackoverflow.com/questions/48769682/how-do-i-convert-data-from-a-scikit-learn-bunch-object-to-a-pandas-dataframe

[2]. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

[3]. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

[4]. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html

[5]. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

[6]. https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

[7]. https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

[8]. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

[9]. Discussed Question 1 and Question 3 with Mohit Dalvi. 